# ACA-Py & ACC-Py Basic Template

## Copy this template into the root folder of your notebook workspace to get started

### Imports

In [1]:
from aries_cloudcontroller import AriesAgentController
import os
from termcolor import colored

from aiohttp import ClientSession

### Initialise the Agent Controller

In [2]:
api_key = os.getenv("ACAPY_ADMIN_API_KEY")
admin_url = os.getenv("ADMIN_URL")

print(f"Initialising a controller with admin api at {admin_url} and an api key of {api_key}")
agent_controller = AriesAgentController(admin_url,api_key)

Initialising a controller with admin api at http://holder-agent:3021 and an api key of adminApiKey


### Start a Webhook Server

In [3]:
webhook_port = int(os.getenv("WEBHOOK_PORT"))
webhook_host = "0.0.0.0"

await agent_controller.init_webhook_server(webhook_host, webhook_port)

print(f"Listening for webhooks from agent at http://{webhook_host}:{webhook_port}")

Listening for webhooks from agent at http://0.0.0.0:3010


In [4]:
listeners = []

# Receive connection messages
def connections_handler(payload):
    state = payload['state']
    connection_id = payload["connection_id"]
    their_role = payload["their_role"]
    routing_state = payload["routing_state"]
    
    print("----------------------------------------------------------")
    print("Connection Webhook Event Received")
    print("Connection ID : ", connection_id)
    print("State : ", state)
    print("Routing State : ", routing_state)
    print("Their Role : ", their_role)
    print("----------------------------------------------------------")

    if state == "invitation":
        # Your business logic
        print("invitation")
    elif state == "request":
        # Your business logic
        print("request")

    elif state == "response":
        # Your business logic
        print("response")

    elif state == "active":
        # Your business logic
        print(colored("Connection ID: {0} is now active.".format(connection_id), "green", attrs=["bold"]))

connection_listener = {
    "handler": connections_handler,
    "topic": "connections"
}

listeners.append(connection_listener)

agent_controller.register_listeners(listeners)

## Establish a Connection

Must establish connection with issuer before being able to receive credential. Holder modeled as invitee in this case. See recipes/connection.

In [5]:
invitation = {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation', '@id': '843d50e1-e861-4ec3-8620-25ae01b2f027', 'label': 'Issuer', 'serviceEndpoint': 'http://issuer-agent:3020', 'recipientKeys': ['8ZjDVHABDMxN973vYVhFifdoz9y3DZYL492aK3mUg1Vg']}

In [6]:
auto_accept=False
alias=None

invite_response = await agent_controller.connections.receive_invitation(invitation, alias, auto_accept)
connection_id = invite_response["connection_id"]

----------------------------------------------------------
Connection Webhook Event Received
Connection ID :  f18b0c81-e8fd-4bfc-83ad-ca1f18efa5d8
State :  invitation
Routing State :  none
Their Role :  inviter
----------------------------------------------------------
invitation
----------------------------------------------------------
Connection Webhook Event Received
Connection ID :  f18b0c81-e8fd-4bfc-83ad-ca1f18efa5d8
State :  request
Routing State :  none
Their Role :  inviter
----------------------------------------------------------
request


In [7]:
response = await agent_controller.connections.create_invitation()

----------------------------------------------------------
Connection Webhook Event Received
Connection ID :  1302de7b-7fe1-4141-bebf-e640a4f71ff7
State :  invitation
Routing State :  none
Their Role :  invitee
----------------------------------------------------------
invitation


In [8]:
print(response["invitation"])

{'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation', '@id': 'abf9fd65-8dae-4875-aea8-31c817bae08b', 'serviceEndpoint': 'http://127.0.0.1:4020', 'recipientKeys': ['Brxr7u7TotYCQJCeXK9BCRdNgi1yYRucLkTRi6siynHU'], 'label': 'Holder'}


## Request Credential From Offer

Note: Your agent will automatically respond if ACAPY_AUTO_RESPOND_CREDENTIAL_OFFER=true flag is set in .env file of agent. Default is false.

To respond to an offer you must identify the offer using the credential_exchange_id generated for it. This is available from within the issue-credential holder handler. You could add custom logic in this loop `elif state == "request":` to handle this.

However, we will fetch the credential exchange records and **assume** this agent only has one record. Customise accordingly.


In [ ]:
# Optional args
thread_id=None
state = "offer_received"
role = "prover"


records_response = await agent_controller.issuer.get_records(connection_id=connection_id,state=state)
record = records_response["results"][0]
record_id = record["credential_exchange_id"]

In [ ]:
await agent_controller.issuer.send_request_for_record(record_id)

## Store Received Credential

This will be done automatically if the ACAPY_AUTO_STORE_CREDENTIAL=true flag is set in the .env file for this agent. Default is false.

Again you could handle this in your holder handler function in the `elif state == "credential_received":` loop.

In [ ]:
# Optionally specify an identifier to uniquely identify this credential within your agents wallet.
# You would likely want to save this somewhere.
# If not set a random one will be generated for you
credential_id = "<UNIQUE CRED ID>"

store_cred_response = await agent_controller.issuer.store_credential(record_id, credential_id)

## Terminate Controller

Whenever you have finished with this notebook, be sure to terminate the controller. This is especially important if your business logic runs across multiple notebooks.

In [ ]:
await agent_controller.terminate()